<a href="https://colab.research.google.com/github/hansa31/Datathon2022/blob/main/DatathonHansa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle


In [1]:
! pwd


/content


In [6]:
cd drive


/content/drive


In [7]:
cd MyDrive/SLIIT_Datathon/Kaggle_H/


/content/drive/.shortcut-targets-by-id/1O9LqeZFL_PcVlvEHq76icalAy5J-FMSA/SLIIT_Datathon/Kaggle_H


In [8]:
pwd


'/content/drive/.shortcut-targets-by-id/1O9LqeZFL_PcVlvEHq76icalAy5J-FMSA/SLIIT_Datathon/Kaggle_H'

In [10]:
# install kaggle API client
! pip install -q kaggle

In [17]:
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [18]:
! chmod 600 ~/.kaggle/kaggle.json

In [22]:

! kaggle datasets download -d ethon0426/lending-club-20072020q1

 97% 469M/482M [00:04<00:00, 107MB/s] 
100% 482M/482M [00:04<00:00, 109MB/s]


In [23]:
pwd

'/content/drive/.shortcut-targets-by-id/1O9LqeZFL_PcVlvEHq76icalAy5J-FMSA/SLIIT_Datathon/Kaggle_H'

In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [41]:
# get the description
def description_getter(col_name):
  des_dic = pd.read_excel('/content/drive/MyDrive/SLIIT_Datathon/Data/LCDataDictionary.xlsx')
  finder = dict(zip(des_dic.LoanStatNew,des_dic.Description))
  print(finder[col_name])

In [45]:
# get the description of the data sets
des_dic = pd.read_excel('/content/drive/MyDrive/SLIIT_Datathon/Data/LCDataDictionary.xlsx')
des_dic

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
148,settlement_amount,The loan amount that the borrower has agreed t...
149,settlement_percentage,The settlement amount as a percentage of the p...
150,settlement_term,The number of months that the borrower will be...
151,NaN,NaN


In [61]:
df1 = pd.read_csv('/content/drive/MyDrive/SLIIT_Datathon/Data/Loan_status_chunk1.csv',index_col=0,low_memory=False)
#df1.head()
#df1.iloc[:,41]

#loan status
df1.loc[:,'loan_status']    #targeted one


id
1077501     Fully Paid
1077430    Charged Off
1077175     Fully Paid
1076863     Fully Paid
1075358     Fully Paid
              ...     
9706491     Fully Paid
9824590     Fully Paid
9824702     Fully Paid
9645390     Fully Paid
9804647     Fully Paid
Name: loan_status, Length: 50000, dtype: object

In [63]:
df1['loan_status'].value_counts()

Fully Paid                                             40414
Charged Off                                             6836
Does not meet the credit policy. Status:Fully Paid      1988
Does not meet the credit policy. Status:Charged Off      761
Name: loan_status, dtype: int64

In [34]:
# Manual exploration
df1.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,...,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,deferral_term,hardship_amount,hardship_length,hardship_dpd,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
count,49999.000000,49999.000000,49999.000000,49999.000000,4.999500e+04,49999.000000,49970.000000,49999.000000,49999.000000,49970.000000,19176.000000,4706.000000,49970.000000,49970.000000,4.999900e+04,49970.000000,49999.0,49999.0,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49854.000000,1727.000000,49999.0,0.0,0.0,0.0,49970.00000,7464.000000,7.464000e+03,0.0,0.0,0.0,...,7464.000000,7464.000000,7464.000000,7464.000000,7464.000000,7464.000000,7464.000000,7464.000000,7464.000000,7216.000000,7464.000000,7464.000000,7464.000000,7464.000000,7391.000000,48634.000000,49894.000000,7.464000e+03,7464.000000,7464.000000,7464.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,8.000000,8.0,8.000000,6.000000,8.000000,8.000000
mean,11644.708894,11416.600332,10835.236129,342.376793,7.020136e+04,13.977421,0.175825,710.100102,714.100122,1.048429,34.705987,63.092435,9.655533,0.075565,1.458287e+04,22.685852,0.0,0.0,12732.626386,12129.149165,10206.357980,2402.561568,1.445089,122.261828,17.284014,2922.295554,690.307926,676.498330,0.000963,42.569774,1.0,NaN,NaN,NaN,0.00056,102.522910,1.441410e+05,NaN,NaN,NaN,...,3.655547,5.711415,4.609191,8.729368,8.661174,8.045016,15.054528,5.702840,11.392551,0.000416,0.002010,0.074625,1.890675,95.073365,51.346994,0.055702,0.006995,1.739145e+05,47219.989014,19447.033092,39206.449893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,131.050000,3.0,7.625000,366.280000,9894.720000,146.115000
std,7676.232456,7477.770715,7513.886540,221.406270,6.197327e+04,7.002816,0.568563,35.797957,35.798033,1.468384,22.277843,44.381499,4.603065,0.303629,2.146495e+04,11.653999,0.0,0.0,9533.302332,9534.509519,7446.502683,2749.391165,8.091599,786.507564,164.093191,4746.882710,82.214462,123.576549,0.032282,20.709453,0.0,NaN,NaN,NaN,0.02530,914.048941,1.539868e+05,NaN,NaN,NaN,...,2.020311,2.899181,2.523081,4.799634,7.121485,3.907837,7.535994,2.899516,4.798664,0.026322,0.044787,0.419834,1.488389,7.497280,34.416025,0.234402,0.131179,1.704049e+05,42146.842558,19064.618193,38750.032077,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,47.359527,0.0,8.667468,135.046549,3981.173652,99.978419
min,500.000000,500.000000,0.000000,15.670000,1.896000e+03,0.000000,0.000000,610.000000,614.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000e+00,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0,NaN,NaN,NaN,0.00000,0.000000,0.000000e+00,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.00000

In [47]:
df1.dtypes

loan_amnt                                     float64
funded_amnt                                   float64
funded_amnt_inv                               float64
term                                           object
int_rate                                       object
                                               ...   
hardship_loan_status                           object
orig_projected_additional_accrued_interest    float64
hardship_payoff_balance_amount                float64
hardship_last_payment_amount                  float64
debt_settlement_flag                           object
Length: 140, dtype: object

In [70]:
# Exploring the dataset --> what feature each column represents

#matrix of dependent variables 
#if the target column is the last one (Debt settlement flag)
x = df1.loc[:,:'loan_status'].values    #need all the rows (:-1)all the columns except the last one
print(x.shape)
print(x)

(50000, 15)
[[5000.0 5000.0 4975.0 ... 'Verified' 'Dec-2011' 'Fully Paid']
 [2500.0 2500.0 2500.0 ... 'Source Verified' 'Dec-2011' 'Charged Off']
 [2400.0 2400.0 2400.0 ... 'Not Verified' 'Dec-2011' 'Fully Paid']
 ...
 [3000.0 3000.0 3000.0 ... 'Not Verified' 'Dec-2013' 'Fully Paid']
 [35000.0 35000.0 35000.0 ... 'Source Verified' 'Dec-2013' 'Fully Paid']
 [17000.0 17000.0 17000.0 ... 'Verified' 'Dec-2013' 'Fully Paid']]


In [71]:
# independent variable (Debt settlement flag)
y = df1.loc[:,'loan_status'].values
y

array(['Fully Paid', 'Charged Off', 'Fully Paid', ..., 'Fully Paid',
       'Fully Paid', 'Fully Paid'], dtype=object)

### Dealing with missing values
*  Remove the lines with data if percentage of missing value is > 50%
* Fill all null variables zero
* Use the Imputerclass from scikit-learn lib to fill in the missing with data's (mean, median, most_frequent)
* fill up the missing value directly after it

### Handling Categorical data
* ML uses numerical values. Therefore, object data type needs to be transformed to numeric.
* Use One Hot Encoder.

In [50]:
# percentage of missing values
def percent_missing(df1):
  percent_missing = df1.isnull().sum() * 100 / len(df1)
  missing_value_df = pd.DataFrame({'column_name': df1.columns,
                                 'percent_missing': percent_missing})
  return missing_value_df

percent_missing(df1)

,column_name,percent_missing
loan_amnt,loan_amnt,0.002
funded_amnt,funded_amnt,0.002
funded_amnt_inv,funded_amnt_inv,0.002
term,term,0.002
int_rate,int_rate,0.002
...,...,...
hardship_loan_status,hardship_loan_status,99.984
orig_projected_additional_accrued_interest,orig_projected_additional_accrued_interest,99.988
hardship_payoff_balance_amount,hardship_payoff_balance_amount,99.984
hardship_last_payment_amount,hardship_last_payment_amount,99.984


In [67]:
# preprocessed df1 with with <50% missing values
def missingPercentagePreProcess(prec,df):
#perc = 50.0
  min_count =  int(((100-perc)/100)*df.shape[0] + 1)
  df_Nomissing = df.dropna(axis=1,thresh = min_count)
  return df_Nomissing
  #percent_missing(df1_Nomissing)

df1_MinMiss = missingPercentagePreProcess(50,df1)

df1_MinMiss


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,debt_settlement_flag
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1077501,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,735.0,739.0,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,May-2020,704.0,700.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1077430,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,car,bike,309xx,GA,1.00,0.0,Apr-1999,740.0,744.0,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,499.0,0.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1077175,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,735.0,739.0,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,739.0,735.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1076863,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,personel,917xx,CA,20.00,0.0,Feb-1996,690.0,694.0,1.0,10.0,0.0,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,604.0,600.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1075358,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,695.0,699.0,0.0,15.0,0.0,27783.0,53.9%,38.0,f,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,Apr-2018,684.0,680.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9706491,30000.0,30000.0,30000.0,60 months,14.47%,705.38,C,C2,Area Director,5 years,MORTGAGE,155000.0,Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,medical bills and debt consolidation,601xx,IL,13.75,0.0,Dec-1994,710.0,714.0,0.0,10.0,0.0,13599.0,51.7%,31.0,w,0.0,0.0,41913.907538,41913.91,30000.00,11913.91,0.00,0.0,0.00,Feb-2018,7350.29,Feb-2018,734.0,730.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
9824590,21000.0,21000.0,21000.0,60 months,15.61%,506.34,C,C4,NaN,NaN,RENT,54000.0,Verified,Dec-2013,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,help wanted,535xx,WI,22.84,1.0,Jun-1996,670.0,674.0,0.0,16.0,0.0,7043.0,87%,24.0,f,0.0,0.0,27293.590003,27293.59,21000.00,6293.59,0.00,0.0,0.00,Mar-2016,14128.75,Aug-2018,714.0,710.0,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
9824702,3000.0,3000.0,3000.0,36 months,13.98%,102.51,C,C

In [ ]:
# dealing with missing values
from sklearn.preprocessing import Imputer